In [1]:
import pandas as pd
import numpy as np
import random
import os
from autogluon.tabular import TabularPredictor
import warnings

# 경고 무시
warnings.filterwarnings('ignore')

# ==========================================
# 0. 시드 고정
# ==========================================
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

# ==========================================
# 1. 데이터 로드
# ==========================================
train_df = pd.read_csv('../Data/train.csv') # 경로 확인 필요
test_df = pd.read_csv('../Data/test.csv')

# ID 컬럼 제거 (학습에 방해됨)
train_df = train_df.drop(['ID'], axis=1)
test_id = test_df['ID'] # 제출용 저장
test_df = test_df.drop(['ID'], axis=1)

target = '임신 성공 여부'

# ==========================================
# 2. 데이터 전처리 (핵심 수정 사항)
# ==========================================

def preprocess_data(df):
    df = df.copy()
    
    # 1. 수치형 변환 (문자열 '회' 제거 등)
    # 기존 코드보다 안전하게 변환
    count_cols = [
        '총 시술 횟수', '클리닉 내 총 시술 횟수', 
        '총 임신 횟수', '총 출산 횟수',
        'IVF 시술 횟수', 'DI 시술 횟수',
        'IVF 임신 횟수', 'DI 임신 횟수',
        'IVF 출산 횟수', 'DI 출산 횟수',
        '총 생성 배아 수', '이식된 배아 수', '미세주입된 난자 수',
        '미세주입에서 생성된 배아 수', '저장된 배아 수'
    ]
    
    for col in count_cols:
        if col in df.columns:
            # 문자열인 경우만 처리
            if df[col].dtype == 'object':
                df[col] = df[col].astype(str).str.replace('회', '')
                df[col] = df[col].astype(str).str.replace(' 이상', '') # "6회 이상" 처리
                # 숫자가 아닌 값(결측 등)은 NaN 처리 후 0으로 채우거나 유지
                df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 2. 나이 매핑 (순서형 변수로 변환)
    age_map = {
        '만18-34세': 0, '만35-37세': 1, '만38-39세': 2, 
        '만40-42세': 3, '만43-44세': 4, '만45-50세': 5, 
        '알 수 없음': 6 # 결측은 별도 범주로
    }
    if '시술 당시 나이' in df.columns:
        df['나이_코드'] = df['시술 당시 나이'].map(age_map).fillna(6)

    # 3. [중요] 결측치가 많은 컬럼을 삭제하지 않고 '정보'로 활용
    # 예: '난자 해동 경과일'이 비어있으면 -> "동결 난자가 아님(신선 난자)"일 확률 높음
    high_nan_cols = ['난자 해동 경과일', '배아 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수']
    
    for col in high_nan_cols:
        if col in df.columns:
            # 값이 있으면 1, 없으면 0인 이진 변수 생성
            df[f'{col}_존재여부'] = np.where(df[col].notnull(), 1, 0)
            # 원본 컬럼의 결측치는 -1 등 특수 값으로 채움 (Tree 모델이 분기하도록)
            df[col] = df[col].fillna(-1)

    return df

# ==========================================
# 3. 파생변수 생성 (Feature Engineering)
# ==========================================

def create_features(df):
    df = df.copy()
    epsilon = 1e-6 # 0 나누기 방지
    
    # A. 성공률 관련 지표 (과거 이력)
    # 0으로 나누는 것 방지 및 시술 횟수가 0인 경우 처리
    if '총 시술 횟수' in df.columns:
        df['임신_성공률'] = df['총 임신 횟수'] / (df['총 시술 횟수'] + 1)
        df['출산_성공률'] = df['총 출산 횟수'] / (df['총 시술 횟수'] + 1)
    
    # B. 배아 퀄리티 추정 지표
    # 생성된 배아 대비 이식된 배아의 비율 (너무 낮으면 배아 질이 안 좋아서 폐기했을 수 있음)
    if '총 생성 배아 수' in df.columns and '이식된 배아 수' in df.columns:
        df['배아_이식_효율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
        
    # C. 미세주입 효율 (ICSI)
    if '미세주입된 난자 수' in df.columns and '미세주입에서 생성된 배아 수' in df.columns:
        df['미세주입_수정률'] = df['미세주입에서 생성된 배아 수'] / (df['미세주입된 난자 수'] + 1)
        
    # D. 나이 보정 이식 수 (나이가 많을수록 이식을 많이 시도하는 경향 보정)
    df['나이_대비_이식수'] = df['나이_코드'] * df['이식된 배아 수']

    # E. 불필요한 컬럼 삭제 (이름 등 식별자나, 이미 인코딩한 원본 문자열)
    # 여기서는 과감하게 drop하지 않고 AutoGluon에게 맡기되, 명확히 불필요한 것만 제거
    drop_candidates = ['시술 시기 코드'] 
    df = df.drop([c for c in drop_candidates if c in df.columns], axis=1)

    return df

# 전처리 적용
train_processed = preprocess_data(train_df)
test_processed = preprocess_data(test_df)

train_processed = create_features(train_processed)
test_processed = create_features(test_processed)

# ==========================================
# 4. 모델 학습 (설정 변경: 과적합 방지)
# ==========================================

# AutoGluon은 결측치와 범주형 변수를 자동으로 잘 처리하므로
# 억지로 One-Hot Encoding을 하거나 NaN을 0으로 다 채울 필요가 없습니다.

predictor = TabularPredictor(
    label=target,
    eval_metric='roc_auc', 
    path='ag_models_final',
    problem_type='binary'
).fit(
    train_data=train_processed,
    # 중요: presets를 'best_quality'로 하되, 스태킹 레벨을 낮춰 과적합을 막습니다.
    presets='best_quality', 
    
    # [핵심 변경 사항]
    # num_stack_levels: 3 -> 1 (과적합 규제)
    # num_bag_folds: 10 (유지, 안정성 확보)
    num_stack_levels=1, 
    num_bag_folds=8,   # 속도와 성능 균형
    
    time_limit=3600,   # 1시간
    
    # LightGBM, CatBoost, XGBoost 등 부스팅 계열이 결측치 처리에 강함
    included_model_types=['GBM', 'CAT', 'XGB', 'RF', 'XT'],
    ag_args_fit={'num_gpus': 1} # GPU 사용 (가능한 경우)
)

# ==========================================
# 5. 결과 확인 및 리더보드 점수 예측
# ==========================================
# 학습 데이터 내에서의 OOF(Out-Of-Fold) 점수 확인 (이 점수가 실제 성능과 유사해야 함)
lb = predictor.leaderboard(train_processed, silent=True)
print(lb[['model', 'score_val']].head())

# ==========================================
# 6. 추론 및 저장
# ==========================================
pred_probs = predictor.predict_proba(test_processed)
# 1번 클래스(성공) 확률
final_probs = pred_probs.iloc[:, 1]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       6.39 GB / 16.00 GB (39.9%)
Disk Space Avail:   183.06 GB / 460.43 GB (39.8%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether 

                     model  score_val
0  RandomForestEntr_BAG_L1   0.729194
1  RandomForestGini_BAG_L1   0.728198
2    ExtraTreesGini_BAG_L1   0.730129
3    ExtraTreesEntr_BAG_L1   0.730319
4      WeightedEnsemble_L2   0.735438


In [4]:
# --- 1. 리더보드 (오름차순 정렬) ---
# 학습 데이터 내에서의 Validation Score 확인
lb = predictor.leaderboard(train_processed, silent=True)
display(lb.head())

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,0.914704,0.729194,roc_auc,1.522996,6.985979,10.704866,1.522996,6.985979,10.704866,1,True,2
1,RandomForestGini_BAG_L1,0.914327,0.728198,roc_auc,1.533938,6.863881,10.202998,1.533938,6.863881,10.202998,1,True,1
2,ExtraTreesGini_BAG_L1,0.897537,0.730129,roc_auc,1.517912,6.805671,9.082332,1.517912,6.805671,9.082332,1,True,3
3,ExtraTreesEntr_BAG_L1,0.896468,0.730319,roc_auc,1.605715,6.986123,9.398705,1.605715,6.986123,9.398705,1,True,4
4,WeightedEnsemble_L2,0.855730,0.735438,roc_auc,9.668214,22.102612,1019.581431,0.006147,0.030373,3.947380,2,True,6


In [5]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_processed.sample(n=min(5000, len(train_df)), random_state=42))
display(fi)

These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인']
Computing feature importance via permutation shuffling for 73 features using 5000 rows with 5 shuffle sets...
	889.68s	= Expected runtime (177.94s per shuffle set)
	115.06s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
배아 이식 경과일,0.031531,0.002166,0.000003,5,0.035991,0.027071
이식된 배아 수,0.028126,0.002562,0.000008,5,0.033400,0.022851
저장된 배아 수,0.014933,0.003618,0.000383,5,0.022382,0.007484
나이_코드,0.011616,0.001215,0.000014,5,0.014118,0.009114
배아_이식_효율,0.003598,0.002325,0.012895,5,0.008384,-0.001188
...,...,...,...,...,...,...
혼합된 난자 수,-0.004002,0.000806,0.999813,5,-0.002342,-0.005662
파트너 정자와 혼합된 난자 수,-0.004492,0.001054,0.999661,5,-0.002322,-0.006662
특정 시술 유형,-0.005245,0.000892,0.999903,5,-0.003408,-0.007082
미세주입된 난자 수,-0.005246,0.000716,0.999959,5,-0.003773,-0.006720


In [7]:
# # --- 3. 제출 파일 생성 ---
from datetime import datetime
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0207_2304_submission.csv에 저장되었습니다.
